# Libraries 

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install datasets --quiet

     |████████████████████████████████| 4.7 MB 41.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.8 MB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 1.2 MB 8.8 MB/s 
     |████████████████████████████████| 365 kB 8.1 MB/s 
     |████████████████████████████████| 141 kB 50.0 MB/s 
     |████████████████████████████████| 212 kB 65.8 MB/s 
     |████████████████████████████████| 127 kB 49.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#libraries for ingesting data
import urllib.request
import pandas as pd
!pip install xmltodict
import xmltodict
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# MedQuAD: Extract Dataset - Beware takes 30 mins


TODO
1. 10 File need custom farsing format - only top 4 with doc format have data
>  MedQuAD/6_NINDS_QA/0000182.xml
MedQuAD/6_NINDS_QA/0000018.xml
MedQuAD/6_NINDS_QA/0000007.xml
MedQuAD/6_NINDS_QA/0000244.xml
MedQuAD/5_NIDDK_QA/0000077.xml
edQuAD/5_NIDDK_QA/0000175.xml
MedQuAD/5_NIDDK_QA/0000065.xml
MedQuAD/5_NIDDK_QA/0000064.xml
MedQuAD/5_NIDDK_QA/0000056.xml
MedQuAD/5_NIDDK_QA/0000177.xml 

2. Takes 30 mins - speed up

In [ ]:
!git clone https://github.com/abachaa/MedQuAD.git

Cloning into 'MedQuAD'...
remote: Enumerating objects: 11301, done.
remote: Total 11301 (delta 0), reused 0 (delta 0), pack-reused 11301
Receiving objects: 100% (11301/11301), 11.00 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (6803/6803), done.


In [ ]:
folders = [x for x in os.walk('MedQuAD')]
len(folders)#30

files = []
for data in folders:
  if data[1] == [] and (data[0].split('/')[1] != '.git'): 
    print(data[0])
    files_in_folder = [f"{data[0]}/{file}"  for file in data[2] if file.split('.')[-1]== 'xml']
    files += files_in_folder
len(files) #

MedQuAD/10_MPlus_ADAM_QA
MedQuAD/11_MPlusDrugs_QA
MedQuAD/7_SeniorHealth_QA
MedQuAD/2_GARD_QA
MedQuAD/8_NHLBI_QA_XML
MedQuAD/12_MPlusHerbsSupplements_QA
MedQuAD/9_CDC_QA
MedQuAD/3_GHR_QA
MedQuAD/1_CancerGov_QA
MedQuAD/4_MPlus_Health_Topics_QA
MedQuAD/6_NINDS_QA
MedQuAD/5_NIDDK_QA


11274

In [ ]:
def convert_xml_to_df(filename):
  #open text file in read mode
  text_file = open(filename, "r")
  #read whole file to a string
  data = text_file.read()
  #close file
  text_file.close()

  testb = bytes(data, 'utf-8')

  data_dict = xmltodict.parse(testb)
  data_frame = pd.DataFrame()
  
  if ('doc' in data_dict.keys()):
    #if  type(data_dict['doc']['qaPairs']) == dict: 
      #data_frame =  pd.json_normalize(data_dict, meta=[['doc', 'qaPairs']])
    print(filename, data_dict)
  elif 'Document' in data_dict.keys():
    if  data_dict['Document']['QAPairs'] is None:
      print(filename, data_dict)
    elif type(data_dict['Document']['QAPairs']['QAPair']) == dict:
      data_frame =  pd.json_normalize(data_dict, meta=[['Document', 'QAPairs']])
    elif  type(data_dict['Document']['QAPairs']['QAPair']) == list:
      data_frame =  pd.json_normalize(data_dict, ['Document', 'QAPairs','QAPair'])
  return data_frame


In [ ]:
full_dataset = pd.DataFrame()
for file in files:
    new_df = convert_xml_to_df(file)
    full_dataset = pd.concat([full_dataset,new_df])
full_dataset.shape #(11263, 19)

MedQuAD/6_NINDS_QA/0000007.xml {'doc': {'@docid': '0000007', '@corpus': 'NINDS', '@url': 'http://www.ninds.nih.gov/disorders/holmes_adie/holmes_adie.htm', 'doctitle-focus': 'Holmes-Adie', 'umls': {'cui': None, 'semanticType': None, 'semanticGroup': None}, 'qaPairs': {'pair': [{'@pid': '1', 'question': {'@qid': '0000007-1', '@qtype': 'information', '#text': 'what is holmes-adie syndrome ?'}, 'answer': 'Holmes-Adie syndrome (HAS) is a neurological disorder affecting the pupil of the eye and the autonomic nervous system.\xa0 It is characterized by one eye with a pupil that is larger than normal and constricts slowly in bright light\xa0 (tonic pupil), along with the absence of deep tendon reflexes, usually in the Achilles tendon.\xa0\xa0 HAS is thought to be the result of a neurotrophic (acting on neurons, or nerve cells) viral infection that causes inflammation and damage to neurons in the ciliary ganglion, an area of the brain that controls eye movements, and the dorsal root ganglion, an

(47436, 23)

In [ ]:
#(47436, 23)
joined_full_dataset = full_dataset[['@pid','Question.@qid',	'Question.@qtype',	'Question.#text', 'Answer',	'Document.QAPairs.QAPair.Answer',
                            'Document.QAPairs.QAPair.@pid',	'Document.QAPairs.QAPair.Question.@qid',	'Document.QAPairs.QAPair.Question.@qtype',	'Document.QAPairs.QAPair.Question.#text']]

joined_full_dataset['pid'] = joined_full_dataset['@pid'].combine_first(joined_full_dataset['Document.QAPairs.QAPair.@pid'])
joined_full_dataset['type'] = joined_full_dataset['Question.@qtype'].combine_first(joined_full_dataset['Document.QAPairs.QAPair.Question.@qtype'])
joined_full_dataset['question'] = joined_full_dataset['Question.#text'].combine_first(joined_full_dataset['Document.QAPairs.QAPair.Question.#text'])
joined_full_dataset['answer'] = joined_full_dataset['Answer'].combine_first(joined_full_dataset['Document.QAPairs.QAPair.Answer'])

joined_full_dataset.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,@pid,Question.@qid,Question.@qtype,Question.#text,Answer,Document.QAPairs.QAPair.Answer,Document.QAPairs.QAPair.@pid,Document.QAPairs.QAPair.Question.@qid,Document.QAPairs.QAPair.Question.@qtype,Document.QAPairs.QAPair.Question.#text,pid,type,question,answer
0,1,0001805-1,information,What is (are) Hand fracture - aftercare ?,None,NaN,NaN,NaN,NaN,NaN,1,information,What is (are) Hand fracture - aftercare ?,NaN
1,2,0001805-2,information,What is (are) Hand fracture - aftercare ?,None,NaN,NaN,NaN,NaN,NaN,2,information,What is (are) Hand fracture - aftercare ?,NaN
2,3,0001805-3,outlook,What is the outlook for Hand fracture - afterc...,None,NaN,NaN,NaN,NaN,NaN,3,outlook,What is the outlook for Hand fracture - afterc...,NaN
3,6,0001805-6,when to contact a medical professional,Do I need to see a doctor for Hand fracture - ...,None,NaN,NaN,NaN,NaN,NaN,6,when to contact a medical professional,Do I need to see a doctor for Hand fracture - ...,NaN
0,NaN,NaN,NaN,NaN,NaN,None,1,0002145-1,information,What is (are) Immunizations - diabetes ?,1,information,What is (are) Immunizations - diabetes ?,None


In [ ]:
joined_full_dataset.to_csv('full_medquad_data.csv')

In [ ]:
joined_full_dataset['answer'].isna().sum() #31034
answered_df = joined_full_dataset[['pid','type','question','answer']][~joined_full_dataset['answer'].isna()]
answered_df['answer'].isna().sum() #0
answered_df.shape #(16402, 4)

In [ ]:
answered_df = answered_df.sample(frac=1).reset_index(drop=True)
rows = answered_df.shape[0]
medal_test_df = answered_df[:int(rows *.2)] #20% test
medal_valid_df = answered_df[int(rows *.2):int(rows *.35)] #15% validation
medal_train_df = answered_df[int(rows *.35):]#65% train

In [ ]:
medal_test_df.to_csv('medal_test.csv')
medal_valid_df.to_csv('medal_valid.csv')
medal_train_df.to_csv('medal_train.csv')

### Full dataset eda

In [ ]:
print(f' number missing answers : {joined_full_dataset["answer"].isna().sum()}') #0
print(f' number missing questions : {joined_full_dataset["question"].isna().sum()}') #0


 number missing answers : 31034
 number missing questions : 0


In [ ]:
answered_df = joined_full_dataset[['pid','type','question','answer']][~joined_full_dataset['answer'].isna()]

print(f' number missing answers : {answered_df["answer"].isna().sum()}') #0



 number missing answers : 0


In [ ]:
print(f' Count of Questions: {len(answered_df.question.unique())}')
print(f' Count of Answers: {len(answered_df.answer.unique())}')


 Count of Questions: 14974
 Count of Answers: 15812


In [ ]:
#Look for questions inside of answers
answer_starts_what = answered_df[answered_df['answer'].str[:5]=="What "]
answer_starts_what[['question', 'answer']]

,question,answer
1,How to prevent Peripheral Arterial Disease (P....,What Causes P.A.D.? The most common cause of P...
2,Who is at risk for Breast Cancer? ?,What Is Cancer Prevention? Cancer prevention i...
3,How to diagnose Skin Cancer ?,What Happens During Screening? Checking for ca...
0,What is (are) High Blood Cholesterol ?,"What is Cholesterol? Cholesterol is a waxy, fa..."
0,What is (are) Kidney Disease ?,What the Kidneys Do You have two kidneys. They...
...,...,...
2,What causes Meesmann corneal dystrophy ?,What causes Meesmann corneal dystrophy? Meesma...
1,What are the symptoms of Loin pain hematuria s...,What are the signs and symptoms of loin pain h...
2,What causes Loin pain hematuria syndrome ?,What causes loin pain hematuria syndrome? The ...
6,What is (are) Kidney Failure: Choosing a Treat...,What should I know about kidney transplantatio...


In [ ]:
#Identify where exact question is replicated in answer

answered_df['length_of_question'] = answered_df['question'].str.len()
answered_df['answer_q_length_substring'] = answered_df.apply(lambda x: x['answer'][:(x['length_of_question'])], axis=1 )

# apply function , questions have space before questionmark, remove when searching
answered_df['Repeat'] = answered_df.apply(lambda x: "yep" if x['answer_q_length_substring'][:-2] ==
                     x['question'][:-2]  else "nope", axis=1)


In [ ]:
a = answered_df[answered_df['Repeat']=="yep"]
a[['question', 'answer']]

,question,answer
2,What causes Klinefelter syndrome ?,What causes Klinefelter syndrome? Klinefelter ...
3,Is Klinefelter syndrome inherited ?,Is Klinefelter syndrome inherited? Klinefelter...
2,What causes Muir-Torre syndrome ?,What causes Muir-Torre syndrome? Muir-Torre sy...
2,Is Kienbock's disease inherited ?,Is Kienbock's disease inherited? There is curr...
2,What causes Mnire's disease ?,What causes Mnire's disease? The underlying ca...
...,...,...
2,What causes Wolff-Parkinson-White syndrome ?,What causes Wolff-Parkinson-White syndrome? No...
2,What causes X-linked adrenal hypoplasia congen...,What causes X-linked adrenal hypoplasia congen...
2,"What causes Ehlers-Danlos syndrome, vascular t...","What causes Ehlers-Danlos syndrome, vascular t..."
3,"Is Ehlers-Danlos syndrome, vascular type inher...","Is Ehlers-Danlos syndrome, vascular type inher..."


In [ ]:
# if repeated then remove question from start of answer
answered_df['clean_answers']= answered_df.apply(lambda x: x['answer'][(x['length_of_question'] -1):] if x['Repeat'] 
                                                == "yep" else x['answer'], axis=1)

In [ ]:
a = answered_df[answered_df['Repeat']=='yep']
a[['question', 'answer', 'clean_answers']]


,question,answer,clean_answers
2,What causes Klinefelter syndrome ?,What causes Klinefelter syndrome? Klinefelter ...,Klinefelter syndrome usually occurs as a rand...
3,Is Klinefelter syndrome inherited ?,Is Klinefelter syndrome inherited? Klinefelter...,"Klinefelter syndrome is not inherited, but us..."
2,What causes Muir-Torre syndrome ?,What causes Muir-Torre syndrome? Muir-Torre sy...,Muir-Torre syndrome is a subtype of Lynch syn...
2,Is Kienbock's disease inherited ?,Is Kienbock's disease inherited? There is curr...,There is currently no evidence that Kienbock'...
2,What causes Mnire's disease ?,What causes Mnire's disease? The underlying ca...,The underlying cause of Mnire's disease is un...
...,...,...,...
2,What causes Wolff-Parkinson-White syndrome ?,What causes Wolff-Parkinson-White syndrome? No...,"Normally, electrical signals in the heart go ..."
2,What causes X-linked adrenal hypoplasia congen...,What causes X-linked adrenal hypoplasia congen...,X-linked adrenal hypoplasia congenita is caus...
2,"What causes Ehlers-Danlos syndrome, vascular t...","What causes Ehlers-Danlos syndrome, vascular t...","Ehlers-Danlos syndrome (EDS), vascular type i..."
3,"Is Ehlers-Danlos syndrome, vascular type inher...","Is Ehlers-Danlos syndrome, vascular type inher...","Ehlers-Danlos syndrome (EDS), vascular type i..."


In [ ]:
answered_df[['pid','type','question','answer']].to_csv('medquad_full_dataset.csv')

In [ ]:
cleaned_repeats = answered_df[['pid','type','question']]
cleaned_repeats['answer'] = answered_df['clean_answers']
cleaned_repeats[['pid','type','question','answer']].to_csv('medquad_full_dataset_no_repeats.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Load Datasets :MedQuAD + TrecLive  - Only need to run this


In [ ]:
# Download trec_live_data.csv from Google Drive
!gdown 1arMHJHkedz-IO98oxwVv9qw8OX5sfK2b -O trec_train.csv
trec_train_df = pd.read_csv('/content/trec_train.csv')
print('Training data shape: ', trec_train_df.shape)
print(trec_train_df.shape)

Downloading...
From: https://drive.google.com/uc?id=1arMHJHkedz-IO98oxwVv9qw8OX5sfK2b
To: /content/trec_train.csv
100% 641k/641k [00:00<00:00, 154MB/s]
Training data shape:  (446, 5)
(446, 5)


In [ ]:

# Download train.csv from Google Drive
!gdown 1du0fmP8WDSW6qh7ESpugmTJ8tEWnSvMR -O medal_train.csv
medal_train_df = pd.read_csv('/content/medal_train.csv')
print('Training data shape: ', medal_train_df.shape)
print(medal_train_df.shape)

Downloading...
From: https://drive.google.com/uc?id=1du0fmP8WDSW6qh7ESpugmTJ8tEWnSvMR
To: /content/medal_train.csv
100% 14.6M/14.6M [00:00<00:00, 50.6MB/s]
Training data shape:  (10662, 5)
(10662, 5)


In [ ]:
# Download valid.csv from Google Drive
!gdown 1bE_9MPsxMqFMlzeW_DCWw_bf2nPL9mw4 -O medal_valid.csv
medal_valid_df = pd.read_csv('/content/medal_valid.csv')
print('Validation data shape: ', medal_valid_df.shape)
print(medal_valid_df.shape)

Downloading...
From: https://drive.google.com/uc?id=1bE_9MPsxMqFMlzeW_DCWw_bf2nPL9mw4
To: /content/medal_valid.csv
100% 3.35M/3.35M [00:00<00:00, 292MB/s]
Validation data shape:  (2460, 5)
(2460, 5)


In [ ]:
# Download medquad_full_dataset.csv from Google Drive
!gdown 1E1j4Iqh82IkeVKdIRl58r2LQDdRgbbaf -O medquad_full_dataset.csv
medquad_df = pd.read_csv('/content/medquad_full_dataset.csv')
print('Training data shape: ', medquad_df.shape)
print(medquad_df[30:40].head(10))

Downloading...
From: https://drive.google.com/uc?id=1E1j4Iqh82IkeVKdIRl58r2LQDdRgbbaf
To: /content/medquad_full_dataset.csv
100% 22.5M/22.5M [00:00<00:00, 109MB/s]
Training data shape:  (16402, 5)
    Unnamed: 0  pid            type  \
30          10   15       treatment   
31          11   16        research   
32          12   17     information   
33           0    1     information   
34           1    2  susceptibility   
35           2    3      prevention   
36           3    4        symptoms   
37           4    8       treatment   
38           5   10     information   
39           6   11     information   

                                             question  \
30  What are the treatments for Peripheral Arteria...   
31  what research (or clinical trials) is being do...   
32  What is (are) Peripheral Arterial Disease (P.A...   
33                           What is (are) Diabetes ?   
34                     Who is at risk for Diabetes? ?   
35                          How

In [ ]:
dia = medquad_df[medquad_df["answer"].str.contains("diabetes")]

In [ ]:
dia.shape

(760, 5)

In [ ]:
dia

,Unnamed: 0,pid,type,question,answer
21,1,2,prevention,How to prevent Peripheral Arterial Disease (P....,What Causes P.A.D.? The most common cause of P...
22,2,3,symptoms,What are the symptoms of Peripheral Arterial D...,Common Symptoms Some people with P.A.D. do not...
23,3,4,exams and tests,How to diagnose Peripheral Arterial Disease (P...,Your Family and Medical History P.A.D. is diag...
26,6,10,susceptibility,Who is at risk for Peripheral Arterial Disease...,Smoking is the main risk factor for P.A.D. You...
27,7,11,symptoms,What are the symptoms of Peripheral Arterial D...,People who have P.A.D. may have symptoms when ...
...,...,...,...,...,...
16352,1,2,causes,What causes Gastroparesis ?,Most people diagnosed with gastroparesis have ...
16354,3,4,exams and tests,How to diagnose Gastroparesis ?,Gastroparesis is diagnosed through a physical ...
16357,6,7,treatment,What are the treatments for Gastroparesis ?,An elevated blood glucose level directly inter...
16358,7,8,information,What is (are) Gastroparesis ?,The problems of gastroparesis can include\n ...


# TREC2017 LiveQA: Extract Data


Researcher Abachaa has partitioned this data and added sub fields on her [repo](https://github.com/abachaa/LiveQA_MedicalTask_TREC2017/blob/master/TrainingDatasets/TREC-2017-LiveQA-Medical-Train-1.xml)

Use researchers data

In [ ]:
!git clone https://github.com/abachaa/LiveQA_MedicalTask_TREC2017.git

Cloning into 'LiveQA_MedicalTask_TREC2017'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 58 (delta 7), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (58/58), done.


### Test dataset

In [ ]:
folders = [x for x in os.walk('LiveQA_MedicalTask_TREC2017/TestDataset')]
test_files = []
for data in folders:
  if data[1] == [] and (data[0].split('/')[1] != '.git'): 
    print(data[0])
    files_in_folder = [f"{data[0]}/{file}"  for file in data[2] if file.split('.')[-1]== 'xml']
    test_files += files_in_folder
len(test_files) #
test_files

LiveQA_MedicalTask_TREC2017/TestDataset


['LiveQA_MedicalTask_TREC2017/TestDataset/TREC-2017-LiveQA-Medical-Test.xml',
 'LiveQA_MedicalTask_TREC2017/TestDataset/TREC-2017-LiveQA-Medical-Test-Questions-w-summaries.xml']

In [ ]:
def trec_convert_test_xml_to_df(filename):
  text_file = open(filename, "r")
  #read whole file to a string
  data = text_file.read()
  #close file
  text_file.close()

  testb = bytes(data, 'utf-8')

  data_dict = xmltodict.parse(testb)
  data_frame = pd.DataFrame()

  data_frame =  pd.json_normalize(data_dict,['LiveQA2017-Medical-Test-Set-Full','NLM-QUESTION'])

  # if answer is empty concatenate all sub answer #text and make answer
  data_frame['sub_answers1'] = data_frame['ReferenceAnswers.RefAnswer'].map(
      lambda x: (' ').join([item['ANSWER'] for item in x]) if type(x)==list else x)
  
  # if answer is empty concatenate all sub answer #text and make answer
  data_frame['sub_answers2'] = data_frame['ReferenceAnswers.ReferenceAnswer'].map(
      lambda x: (' ').join([item['ANSWER'] for item in x]) if type(x)==list else x)

  sub_answer_column = 'ReferenceAnswers.ReferenceAnswer.ANSWER' 
  #question_column = '@questionid' if '@questionid' in data_frame.columns else '@qid'
  data_frame['answer'] = data_frame[sub_answer_column].combine_first(data_frame['sub_answers1'])
  data_frame['answer'] = data_frame['answer'].combine_first(data_frame['sub_answers2'])

  #data_frame['answer'] = data_frame.apply(lambda x: x['sub_answers']if x[sub_answer_column].isna else x[sub_answer_column])
  #rename columns for readability
  data_frame['pid'] = data_frame['Original-Question.@qfile']
  data_frame['type'] = data_frame['Original-Question.SUBJECT']
  data_frame['question'] = data_frame['Original-Question.MESSAGE']

  return data_frame[['pid', 'type', 'question', 'answer']]


trec_full_test_dataset = pd.DataFrame()
#overwrite files list to only hold one file as the summary file has the same information and more that is not used for this research
files = ['LiveQA_MedicalTask_TREC2017/TestDataset/TREC-2017-LiveQA-Medical-Test.xml']
for file in files:
    new_df = trec_convert_test_xml_to_df(file)
    trec_full_test_dataset = pd.concat([trec_full_test_dataset,new_df])
trec_full_test_dataset.shape #(446, 19)

(104, 4)

In [ ]:
filename ='LiveQA_MedicalTask_TREC2017/TestDataset/TREC-2017-LiveQA-Medical-Test.xml'

text_file = open(filename, "r")
#read whole file to a string
data = text_file.read()
#close file
text_file.close()

testb = bytes(data, 'utf-8')

data_dict = xmltodict.parse(testb)
data_frame = pd.DataFrame()


In [ ]:

data_frame =  pd.json_normalize(data_dict,['LiveQA2017-Medical-Test-Set-Full','NLM-QUESTION'])
data_frame.head(4)

,@qid,NIST-PARAPHRASE,Original-Question.@qfile,Original-Question.SUBJECT,Original-Question.MESSAGE,ANNOTATIONS.FOCUS,ANNOTATIONS.TYPE.@tid,ANNOTATIONS.TYPE.@hasFocus,ANNOTATIONS.TYPE.#text,ReferenceAnswers.RefAnswer,...,ANNOTATIONS.KEYWORD.@kid,ANNOTATIONS.KEYWORD.@kcategory,ANNOTATIONS.KEYWORD.#text,ReferenceAnswers.ReferenceAnswer.@aid,ReferenceAnswers.ReferenceAnswer.ANSWER,ReferenceAnswers.ReferenceAnswer.AnswerURL,ReferenceAnswers.ReferenceAnswer.COMMENT,ReferenceAnswers.ReferenceAnswer,ANNOTATIONS.TYPE,ANNOTATIONS.KEYWORD
0,TQ1,What is the relationship between Noonan syndro...,71.txt,Noonan syndrome,What are the references with noonan syndrome a...,"[{'@fid': 'F1', '@fcategory': 'Problem', '#tex...",T1,"F1,F2",EFFECT,"[{'@aid': 'TQ1A1', 'ANSWER': 'Noonan's syndrom...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TQ2,Do 5 mg. Zolmitriptan tabkets contain gluten?,NaN,Gluten information,Re:NDC# 0115-0672-50 Zolmitriptan tabkets 5mg....,NaN,T1,F1,INGREDIENT,NaN,...,K1,Substance,gluten,TQ2A1,Zolmitriptan tablets are available as 2.5 mg (...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,A list of ingredients is a relevant answer.,NaN,NaN,NaN
2,TQ3,Are amphetamine salts of 20 mg dosage gluten f...,1-131219913.xml.txt,amphetamine salts 20 mg,are they gluten free,NaN,T1,F1,INGREDIENT,NaN,...,K1,Substance,gluten,NaN,NaN,NaN,NaN,"[{'@aid': 'TQ3A1', 'ANSWER': 'Active Ingredien...",NaN,NaN
3,TQ4,What are the treatments and precautions for VD...,NF_315.txt,vdrl positive,vdrl positive patients please tell me what are...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TQ4A1,"Syphilis\n\t\t\t\t\tIf the RPR, VDRL, or TRUST...",https://medlineplus.gov/ency/article/000861.htm,Answers about diagnostic methods after a posit...,NaN,"[{'@tid': 'T1', '@hasFocus': 'F1', '#text': 'T...",NaN


In [ ]:
data_dict

In [ ]:
trec_full_test_dataset[:3]

,pid,type,question,answer
0,71.txt,Noonan syndrome,What are the references with noonan syndrome a...,Noonan's syndrome is an eponymic designation t...
1,NaN,Gluten information,Re:NDC# 0115-0672-50 Zolmitriptan tabkets 5mg....,Zolmitriptan tablets are available as 2.5 mg (...
2,1-131219913.xml.txt,amphetamine salts 20 mg,are they gluten free,Active Ingredients\n\t\t\t\t\tAmphetamine Aspa...


In [ ]:
trec_full_test_dataset.shape #(104, 4)
test_has_answers = trec_full_test_dataset[trec_full_test_dataset['answer'].notnull()]
test_has_answers.shape #(104, 4)
test_has_qa = test_has_answers[test_has_answers['question'].notnull()]
test_has_qa.shape #(104, 4)
trec_full_test_dataset = test_has_qa

(104, 4)

In [ ]:
trec_full_test_dataset.to_csv('trec_live_data_test.csv')

### Train dataset

In [ ]:
filename = 'LiveQA_MedicalTask_TREC2017/TrainingDatasets/TREC-2017-LiveQA-Medical-Train-1.xml'
#open text file in read mode
text_file = open(filename, "r")
#read whole file to a string
data = text_file.read()
#close file
text_file.close()

testb = bytes(data, 'utf-8')

data_dict = xmltodict.parse(testb)
data_frame = pd.DataFrame()

data_frame =  pd.json_normalize(data_dict,['NLM-QUESTIONS','NLM-QUESTION'])

In [ ]:
data_frame[:5]

,@questionid,@fRef,SUBJECT,MESSAGE,SUB-QUESTIONS.SUB-QUESTION.@subqid,SUB-QUESTIONS.SUB-QUESTION.ANNOTATIONS.FOCUS,SUB-QUESTIONS.SUB-QUESTION.ANNOTATIONS.TYPE,SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER,SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER.@answerid,SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER.@pairid,SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER.#text,SUB-QUESTIONS.SUB-QUESTION
0,Q1,11373,None,Literature on Cardiac amyloidosis. Please let...,Q1-S1,cardiac amyloidosis,information,"[{'@answerid': 'Q1-S1-A1', '@pairid': '1', '#t...",NaN,NaN,NaN,NaN
1,Q2,1-136434885,treatment options versus migraine types,Migraine seems to be a spectrum of conditions ...,Q2-S1,migraine,treatment,"[{'@answerid': 'Q2-S1-A1', '@pairid': '3', '#t...",NaN,NaN,NaN,NaN
2,Q3,1-123453375,None,DO I USE PYRIDOXINE TABLETS EVEN IF IM PREGNANT?,Q3-S1,pyridoxine,contraindication,NaN,Q3-S1-A1,5,"Before taking pyridoxine, tell your doc...",NaN
3,Q4,1-118260435,cramp,i have lymphoma what causes cramp after chemo ...,Q4-S1,cramp,cause,NaN,Q4-S1-A1,6,Muscle cramps are common and often occur when ...,NaN
4,Q5,1-132032443,Retina,I wonder of new research and testing on macula...,Q5-S1,macular degeneration,treatment,NaN,Q5-S1-A1,7,These resources address the diagnosis or manag...,NaN


In [ ]:
folders = [x for x in os.walk('LiveQA_MedicalTask_TREC2017/TrainingDatasets')]

In [ ]:

files = []
for data in folders:
  if data[1] == [] and (data[0].split('/')[1] != '.git'): 
    print(data[0])
    files_in_folder = [f"{data[0]}/{file}"  for file in data[2] if file.split('.')[-1]== 'xml']
    files += files_in_folder
len(files) #
files

LiveQA_MedicalTask_TREC2017/TrainingDatasets


['LiveQA_MedicalTask_TREC2017/TrainingDatasets/TREC-2017-LiveQA-Medical-Train-1.xml',
 'LiveQA_MedicalTask_TREC2017/TrainingDatasets/TREC-2017-LiveQA-Medical-Train-2.xml']

In [ ]:
def trec_convert_xml_to_df(filename):
  #open text file in read mode
  text_file = open(filename, "r")
  #read whole file to a string
  data = text_file.read()
  #close file
  text_file.close()

  testb = bytes(data, 'utf-8')

  data_dict = xmltodict.parse(testb)
  data_frame = pd.DataFrame()
  
  data_frame =  pd.json_normalize(data_dict,['NLM-QUESTIONS','NLM-QUESTION'])
  # if answer is empty concatenate all sub answer #text and make answer
  data_frame['sub_answers'] = data_frame['SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER'].map(
      lambda x: (' ').join([item['#text'] for item in x]) if type(x)==list else x)

  sub_answer_column = 'SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER.#text' if 'SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER.#text' in data_frame.columns else 'SUB-QUESTIONS.SUB-QUESTION.ANSWERS.ANSWER'
  question_column = '@questionid' if '@questionid' in data_frame.columns else '@qid'
  data_frame['answer'] = data_frame[sub_answer_column].combine_first(data_frame['sub_answers'])
  #rename columns for readability
  data_frame['pid'] = data_frame[question_column]
  data_frame['type'] = data_frame['SUBJECT']
  data_frame['question'] = data_frame['MESSAGE']

  return data_frame[['pid', 'type', 'question', 'answer']]



trec_full_dataset = pd.DataFrame()
for file in files:
    new_df = trec_convert_xml_to_df(file)
    trec_full_dataset = pd.concat([trec_full_dataset,new_df])
trec_full_dataset.shape #(446, 19)

(446, 4)

In [ ]:

trec_full_dataset[['pid', 'type', 'question', 'answer']]

,pid,type,question,answer
0,Q1,None,Literature on Cardiac amyloidosis. Please let...,Cardiac amyloidosis is a disorder caused by de...
1,Q2,treatment options versus migraine types,Migraine seems to be a spectrum of conditions ...,There is no specific cure for migraine headach...
2,Q3,None,DO I USE PYRIDOXINE TABLETS EVEN IF IM PREGNANT?,"Before taking pyridoxine, tell your doc..."
3,Q4,cramp,i have lymphoma what causes cramp after chemo ...,Muscle cramps are common and often occur when ...
4,Q5,Retina,I wonder of new research and testing on macula...,These resources address the diagnosis or manag...
...,...,...,...,...
241,316.0,MRSA,If you should contact MRSA and take antibiotic...,"Occasionally, a person who has been treated fo..."
242,317.0,intraductal papilloma,What if you do not have surgery to remove a pa...,Your specialist will be able to advise you of ...
243,318.0,Excessive Eye Water,Thank for your information on my earlier quest...,Your doctor may order tests to help determine ...
244,319.0,None,I have numbness/tingling in my lower right arm...,Home Care Your doctor should find and treat t...


In [ ]:
trec_train_df.shape #(446, 4)
has_answers = trec_train_df[trec_train_df['answer'].notnull()]
has_answers.shape #(411, 4)
has_qa = has_answers[has_answers['question'].notnull()]
has_qa.shape #(409, 4)
trec_train_df = has_qa

(409, 5)

In [ ]:
trec_full_dataset.to_csv('trec_live_data.csv')

### Load Trec Datasets - updated to cols [pid | type | q | a] 

In [ ]:
# Download trec_live_data.csv from Google Drive
!gdown 14bnH4InS57HfnT9hmimg477yFAUdDcAn -O trec_train.csv
trec_train_df = pd.read_csv('/content/trec_train.csv')
print('Training data shape: ', trec_train_df.shape)
print(trec_train_df.head(2))

Downloading...
From: https://drive.google.com/uc?id=14bnH4InS57HfnT9hmimg477yFAUdDcAn
To: /content/trec_train.csv
100% 628k/628k [00:00<00:00, 81.2MB/s]
Training data shape:  (409, 5)
   Unnamed: 0 pid                                     type  \
0           0  Q1                                      NaN   
1           1  Q2  treatment options versus migraine types   

                                            question  \
0  Literature on Cardiac amyloidosis.  Please let...   
1  Migraine seems to be a spectrum of conditions ...   

                                              answer  
0  Cardiac amyloidosis is a disorder caused by de...  
1  There is no specific cure for migraine headach...  


In [ ]:
# Download trec_live_data.csv from Google Drive
!gdown 1Wag9dJAK207G15l4DHz_vGkYhJuLtYK0 -O trec_test.csv
trec_test_df = pd.read_csv('/content/trec_test.csv')
print('Test data shape: ', trec_test_df.shape)
print(trec_test_df.head(3))

Downloading...
From: https://drive.google.com/uc?id=1Wag9dJAK207G15l4DHz_vGkYhJuLtYK0
To: /content/trec_test.csv
100% 157k/157k [00:00<00:00, 90.0MB/s]
Test data shape:  (104, 5)
   Unnamed: 0                  pid                     type  \
0           0               71.txt          Noonan syndrome   
1           1                  NaN       Gluten information   
2           2  1-131219913.xml.txt  amphetamine salts 20 mg   

                                            question  \
0  What are the references with noonan syndrome a...   
1  Re:NDC# 0115-0672-50 Zolmitriptan tabkets 5mg....   
2                               are they gluten free   

                                              answer  
0  Noonan's syndrome is an eponymic designation t...  
1  Zolmitriptan tablets are available as 2.5 mg (...  
2  Active Ingredients\n\t\t\t\t\tAmphetamine Aspa...  
